# PyCOMPSs: Polymorphism

In this example we will how to use polimorphism with PyCOMPSs.

## Import the PyCOMPSs library

In [ ]:
import pycompss.interactive as ipycompss

## Start the runtime

Initialize COMPSs runtime Parameters indicates if the execution will generate task graph and sets the monitor interval.

In [ ]:
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True, debug=False,
                    project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000)

## Create a file to define the tasks

Importing task, implement and constraint modules 

In [ ]:
%%writefile module.py

from pycompss.api.task import task
from pycompss.api.implement import implement
from pycompss.api.constraint import constraint

## Declaring tasks into the file

Declare functions and decorate with @task those that should be tasks 

In [ ]:
%%writefile -a module.py

@constraint(computing_units='1')
@task(returns=list)
def addtwovectors(list1, list2):
    for i in range(len(list1)):
        list1[i] += list2[i]
    return list1

In [ ]:
%%writefile -a module.py

@implement(source_class="module", method="addtwovectors")
@constraint(computing_units='4')
@task(returns=list)
def addtwovectorsWithNumpy(list1, list2):
    import numpy as np
    x = np.array(list1)
    y = np.array(list2)
    z = x + y
    return z.tolist()

## Invoking tasks 

In [ ]:
from pycompss.api.api import compss_wait_on
from module import addtwovectors  # Just import and use addtwovectors
from random import random

vectors = 100
vector_length = 5000
vectors_a = [[random() for i in range(vector_length)] for i in range(vectors)]
vectors_b = [[random() for i in range(vector_length)] for i in range(vectors)]

results = []
for i in range(vectors):
    results.append(addtwovectors(vectors_a[i], vectors_b[i]))

### <font color='green'> Accessing data outside tasks requires synchronization </green>

In [ ]:
results = compss_wait_on(results)
print(len(results))
print(results[0])

## Stop the runtime

In [ ]:
ipycompss.stop(sync=True)